# Projet 3 - Anticipez les besoins en consommation de bâtiments

## Notebook de selections des variables importantes

Le but de ce notebook est d'utiliser le dataset clean généré par l'analyse exploratoire, et de créer des modèles prédictifs pour les consommations énergétiques et l'émission de CO2.

In [ ]:
import pandas as pd
from MLUtils import DataAnalysis

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Importation du jeu de données
df = pd.read_csv('data/clean.csv')

In [ ]:
df.info()

In [ ]:
df.describe()

#### Normalisation des données avec MinMaxScaler

In [ ]:
from sklearn.preprocessing import MinMaxScaler
import pandas as pd

# Sélection des colonnes numériques
numeric_columns = df.select_dtypes(include=['int64', 'float64']).columns

# Sélection des colonnes non numériques
non_numeric_columns = df.select_dtypes(exclude=['int64', 'float64']).columns

# Création du scaler
scaler = MinMaxScaler()

# Application de la normalisation sur les colonnes numériques
scaled_numeric_data = scaler.fit_transform(df[numeric_columns])

# Création d'un DataFrame pour les données normalisées
df_scaled_numeric = pd.DataFrame(scaled_numeric_data, columns=numeric_columns)

# Combinaison des données numériques normalisées avec les données non numériques
df_scaled = pd.concat([df_scaled_numeric, df[non_numeric_columns].reset_index()], axis=1)

# Affichage des premières lignes pour vérifier la création de df_scaled
df_scaled.head()



In [ ]:
df_scaled.info()

## Nous avons maintenant un dataframe contenant des colonnes normalisées.

Nous allons maintenant nous intéresser à la colonne energystarscore.

Notre objectif sera de voir ses correlations avec les autres colonnes, et de décider s'il convient de la conserver ou non selon ce critère.

In [ ]:
# Remove lines with an empty ENERGYSTARScore
df_scaled = df_scaled.dropna(subset=['ENERGYSTARScore'])

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Séparation des caractéristiques et des cibles
X_with_energy_star = df_scaled.drop(['SiteEnergyUse(kBtu)', 'TotalGHGEmissions'], axis=1)
X_without_energy_star = df_scaled.drop(['SiteEnergyUse(kBtu)', 'TotalGHGEmissions', 'ENERGYSTARScore'], axis=1)

# Cibles
y_site_energy_use = df_scaled['SiteEnergyUse(kBtu)']
y_total_ghg_emissions = df_scaled['TotalGHGEmissions']




In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Séparation des ensembles d'apprentissage et de test
X_train_with, X_test_with, y_train, y_test = train_test_split(X_with_energy_star, y_site_energy_use, test_size=0.2, random_state=42)
X_train_without, X_test_without, _, _ = train_test_split(X_without_energy_star, y_site_energy_use, test_size=0.2, random_state=42)

# 1) Features correlation

# 2 ) Entrainement des modèles

### A) Modèles linéaires

#### i) Régression linéaire

In [ ]:
import time

# Avec ENERGYSTARScore

lin_reg_with_model = LinearRegression()

start = time.time()
lin_reg_with_model.fit(X_train_with, y_train)
end = time.time()

time_for_training_lin_reg_with = end - start

# print the time for training
print('Time for training with ENERGYSTARScore: ', time_for_training_lin_reg_with)

y_pred_lin_reg_with = lin_reg_with_model.predict(X_test_with)

mse__lin_reg_with = mean_squared_error(y_test, y_pred_lin_reg_with)

In [ ]:
# save the model as a pickle file
import pickle
with open('lin_reg_with_energy_star.pkl', 'wb') as file:
    pickle.dump(lin_reg_with_model, file)

# obtain the size on the disk of this model
import os
size_lin_reg_with = os.path.getsize('lin_reg_with_energy_star.pkl')

# tranform into KB
size_lin_reg_with = size_lin_reg_with / 1024

# print the size of the model
print('Taille du modele avec ENERGYSTARScore: ', size_lin_reg_with, 'Ko')

In [ ]:





# Sans ENERGYSTARScore
lin_reg_without_model = LinearRegression().fit(X_train_without, y_train)
y_pred_without = lin_reg_without_model.predict(X_test_without)
mse_without = mean_squared_error(y_test, y_pred_without)

print(f"MSE avec ENERGYSTARScore: {mse_with}, sans ENERGYSTARScore: {mse_without}")

from sklearn.tree import DecisionTreeRegressor

# Avec ENERGYSTARScore
tree_reg_with_model = DecisionTreeRegressor(random_state=42).fit(X_train_with, y_train)
y_pred_with = tree_reg_with_model.predict(X_test_with)
mse_with = mean_squared_error(y_test, y_pred_with)

# Sans ENERGYSTARScore
tree_reg_without = DecisionTreeRegressor(random_state=42).fit(X_train_without, y_train)
y_pred_without = tree_reg_without.predict(X_test_without)
mse_without = mean_squared_error(y_test, y_pred_without)

print(f"MSE avec ENERGYSTARScore: {mse_with}, sans ENERGYSTARScore: {mse_without}")


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Séparation des ensembles d'apprentissage et de test
X_train_with, X_test_with, y_train, y_test = train_test_split(X_with_energy_star, y_total_ghg_emissions, test_size=0.2, random_state=42)
X_train_without, X_test_without, _, _ = train_test_split(X_without_energy_star, y_total_ghg_emissions, test_size=0.2, random_state=42)

# Avec ENERGYSTARScore
lin_reg_with_model = LinearRegression().fit(X_train_with, y_train)
y_pred_with = lin_reg_with_model.predict(X_test_with)
mse_with = mean_squared_error(y_test, y_pred_with)

# Sans ENERGYSTARScore
lin_reg_without_model = LinearRegression().fit(X_train_without, y_train)
y_pred_without = lin_reg_without_model.predict(X_test_without)
mse_without = mean_squared_error(y_test, y_pred_without)

print(f"MSE avec ENERGYSTARScore: {mse_with}, sans ENERGYSTARScore: {mse_without}")

from sklearn.tree import DecisionTreeRegressor

# Avec ENERGYSTARScore
tree_reg_with_model = DecisionTreeRegressor(random_state=42).fit(X_train_with, y_train)
y_pred_with = tree_reg_with_model.predict(X_test_with)
mse_with = mean_squared_error(y_test, y_pred_with)

# Sans ENERGYSTARScore
tree_reg_without = DecisionTreeRegressor(random_state=42).fit(X_train_without, y_train)
y_pred_without = tree_reg_without.predict(X_test_without)
mse_without = mean_squared_error(y_test, y_pred_without)

print(f"MSE avec ENERGYSTARScore: {mse_with}, sans ENERGYSTARScore: {mse_without}")

### Nous constatons, selon notre analyse, que la colonne ENERGYSTARScore ne semble pas avoir d'importance, ni pour l'estimation de TotalEnergyUse, ni pour celui de TotalGHGEmission, que ce soit dans le cas d'une utilisation d'un modèle linéaire ou d'un modèle non-linéaire. En effet les valeurs MSE sont les mêmes avec ou sans cette colonne. Nous allons donc supprimer cette colonne pour la suite de notre analyse de modèles.

In [ ]:
# remove the ENERGYSTARScore column
df_scaled_without_energystarscore = df_scaled.drop('ENERGYSTARScore', axis=1)

# drop the index column
df_scaled_without_energystarscore = df_scaled_without_energystarscore.drop('index', axis=1)

# 3) tableau récapitulatif des modèles

| Modèle | MSE TotalEnergyUse | MSE TotalGHGEmission | Temps d'entrainement | Taille du modèle |
| --- | --- | --- | --- | --- |
| Régression linéaire | 0.000 | 0.000 | 0.000 | 0.000 |
| Random Forest | 0.000 | 0.000 | 0.000 | 0.000 |

On remarque qu'on obtient les meilleurs résultats avec le modèle <rfr>. Même si ce modèle est plus volumineux, il est plus performant. Nous allons donc le conserver pour la suite de notre analyse.

# 4) Feature importance locale et globale (SHAP)

## Correlation des variables

Nous allons maintenant faire un tableau de correlation en utilisant la méthode de Pearson des variables afin d'observer si les colonnes cibles sont correlés (ce que nous suspectons fortement)

In [ ]:
import pandas as pd
import numpy as np

# Calcul de la matrice de corrélation
correlation_matrix = df_scaled_without_energystarscore.corr(method='pearson')

correlation_matrix


In [ ]:
import plotly.figure_factory as ff

# Création de la figure Plotly pour la grande matrice de corrélation
fig_large = ff.create_annotated_heatmap(
    z=correlation_matrix.values,
    x=correlation_matrix.columns.tolist(),
    y=correlation_matrix.columns.tolist(),
    colorscale='Viridis',
    annotation_text=correlation_matrix.round(2).values,
    showscale=True
)

# Ajustement des dimensions de la figure pour améliorer l'affichage
fig_large.update_layout(
    title_text='Matrice de corrélation agrandie',
    title_x=0.5,
    width=1800, # Ajustement de la largeur
    height=900, # Ajustement de la hauteur
    autosize=False
)

# Affichage de la figure
fig_large.show()



#### Nous constatons que les colonnes TotalGHGEmission et SiteEnergyUse sont comme prévu très correllées. Nous allons donc les éliminer pour établir les modèles de prédiction de l'une ou de l'autre.

Nous observons également que les colonnes suivantes sont fortement correllées :
- SiteEUI et SiteEUIWN
- SourceEUI et SourceEUIWN
- SiteEnergyUse et SiteEnergyUseWN
- NaturalGas(Kbtu) et NaturalGas(therms)

Nous allons donc supprimer les colonnes WN ainsi que NaturalGas(therms)

In [ ]:
# drop columns SiteEUIWN(kBtu/sf), SourceEUIWN(kBtu/sf), SiteEnergyUseWN(kBtu), NaturalGas(therms)
df_scaled_without_energystarscore = df_scaled_without_energystarscore.drop(['SiteEUIWN(kBtu/sf)', 'SourceEUIWN(kBtu/sf)', 'SiteEnergyUseWN(kBtu)', 'NaturalGas(therms)'], axis=1)

# Analyse des features importance locale avec SHAP pour le SiteEnergyUse(kBtu)

In [ ]:

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import shap

# Séparation des caractéristiques et de la cible
X = df_scaled_without_energystarscore.drop(['SiteEnergyUse(kBtu)', 'TotalGHGEmissions'], axis=1)
y = df_scaled_without_energystarscore['SiteEnergyUse(kBtu)']

# Division en ensemble d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=62)

# Entraînement du modèle
rfr_model = RandomForestRegressor(n_estimators=100,random_state=42)
rfr_model.fit(X_train, y_train)

# Calcul des valeurs SHAP
explainer = shap.Explainer(rfr_model, X_train)
shap_values = explainer(X_test)

# Visualisation
shap.summary_plot(shap_values, X_test)

# Analyse des features importance locale avec SHAP pour le TotalGHGEmissions

In [ ]:

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import shap

# Séparation des caractéristiques et de la cible
X = df_scaled_without_energystarscore.drop(['SiteEnergyUse(kBtu)', 'TotalGHGEmissions'], axis=1)
y = df_scaled_without_energystarscore['TotalGHGEmissions']

# Division en ensemble d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=62)

# Entraînement du modèle
rfr_model = RandomForestRegressor(n_estimators=100,random_state=42)
rfr_model.fit(X_train, y_train)

# Calcul des valeurs SHAP
explainer = shap.Explainer(rfr_model, X_train)
shap_values = explainer(X_test)

# Visualisation
shap.summary_plot(shap_values, X_test)

In [ ]:
y_pred = rfr_model.predict(X_test)

In [ ]:
# Find the index of the minimum value and the maximum value
min_index = np.argmin(y_pred)
max_index = np.argmax(y_pred)

In [ ]:
# visualize the first prediction's explanation
shap.initjs()
shap.plots.force(shap_values[min_index, ...])

In [ ]:
# visualize the first prediction's explanation
shap.initjs()
shap.plots.force(shap_values[max_index, ...])

In [ ]:
# visualize the first prediction's explanation
shap.initjs()
shap.plots.force(shap_values[0, ...])

In [ ]:
shap.plots.force(shap_values[15, ...])

## Features importance globales pour le calcul du SiteEnergyUse

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import plotly.express as px



y = df_scaled_without_energystarscore['SiteEnergyUse(kBtu)']  # Supposition pour l'exemple
X = df_scaled_without_energystarscore.drop(columns=['SiteEnergyUse(kBtu)', 'TotalGHGEmissions'])

# Division en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entraînement du modèle RandomForestRegressor
rfr_model = RandomForestRegressor(n_estimators=100, random_state=42)
rfr_model.fit(X_train, y_train)

# Obtention de l'importance des caractéristiques
feature_importances = rfr_model.feature_importances_
features = X.columns
importances_df = pd.DataFrame({'Feature': features, 'Importance': feature_importances})

# Visualisation de l'importance des caractéristiques avec Plotly
fig = px.bar(importances_df.sort_values('Importance', ascending=False), 
             x='Importance', 
             y='Feature', 
             height=800,
             title="Feature Importances using RandomForestRegressor",
             labels={'Feature':'Feature', 'Importance':'Importance'},
             orientation='h')
fig.show()


# Nos analyses des features importances locales et globales nous permettent de choisir les variables importantes

In [ ]:
# create a new dataframes with columns that have an importance greater than 0.01
# columns 'NaturalGas(kBtu)', 'SteamUse(kBtu)', 'GHGEmissionsIntensity', 'NumberofBuildings', 'NumberofFloors', 'Age', 'SiteEUI(kBtu/sf)', 'SourceEUI(kBtu/sf)', 'SiteEnergyUse(kBtu)'
df_SEU = df_scaled_without_energystarscore[['NaturalGas(kBtu)', 'SteamUse(kBtu)', 'GHGEmissionsIntensity', 'NumberofBuildings', 'NumberofFloors', 'Age', 'SiteEUI(kBtu/sf)', 'SourceEUI(kBtu/sf)', 'SiteEnergyUse(kBtu)']]

## Features importance pour le calcul du TotalGHGEmissions

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import plotly.express as px



y = df_scaled_without_energystarscore['TotalGHGEmissions']  # Supposition pour l'exemple
X = df_scaled_without_energystarscore.drop(columns=['TotalGHGEmissions', 'SiteEnergyUse(kBtu)'])

# Division en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entraînement du modèle RandomForestRegressor
rfr_model = RandomForestRegressor(n_estimators=100, random_state=42)
rfr_model.fit(X_train, y_train)

# Obtention de l'importance des caractéristiques
feature_importances = rfr_model.feature_importances_
features = X.columns
importances_df = pd.DataFrame({'Feature': features, 'Importance': feature_importances})

# Visualisation de l'importance des caractéristiques avec Plotly
fig = px.bar(importances_df.sort_values('Importance', ascending=False), 
             x='Importance', 
             y='Feature', 
             height=800,
             title="Feature Importances using RandomForestRegressor",
             labels={'Feature':'Feature', 'Importance':'Importance'},
             orientation='h')
fig.show()


# Nos analyses des features importances locales et globales nous permettent de choisir les variables importantes

In [ ]:
# create a new dataframe, with data from df_scaled_without_energystarscore with columns that have an importance greater than 0.01

df_TGE = df_scaled_without_energystarscore[['NaturalGas(kBtu)', 'SteamUse(kBtu)', 'GHGEmissionsIntensity', 'NumberofBuildings', 'NumberofFloors', 'Age', 'SiteEUI(kBtu/sf)', 'TotalGHGEmissions']]

## Nous avons maintenant nos deux datasets sur lesquels nous allons créer nos modèles
- Pour SiteEnergyUse, nous avons le dataset df_SEU
- Pour TotalGHGEmissions, nous avons le data df_TGE

In [ ]:
DataAnalysis.show_columns_population(df_SEU, type="bar")

In [ ]:
DataAnalysis.show_columns_population(df_TGE, type="bar")

# Nos datasets sont prêts pour l'exploration des modèles consécutive. Il n'y a pas de valeur manquante, les valeurs sont normalisées, nous avons pris en compte l'importance des variables et leur correlations

## Nous avons 2524 observations pour chacun des datasets.

In [ ]:
df_TGE.to_csv('data/df_TGE.csv', index=False)
df_SEU.to_csv('data/df_SEU.csv', index=False)